In [1]:
# Roamio - AI Agent for International Student Housing (Updated with Kaggle Dataset)
!pip install gradio requests beautifulsoup4
!pip install chromadb
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 38.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28

In [21]:
import gradio as gr
import json
import re
import requests
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb import PersistentClient

MAX_RESULTS = 5
SERPER_API_KEY = "77420f146d32150c69fa44ab1d0f7f22983e36c8"  # Replace with your key from serper.dev

def extract_price_range(query):
    """Extract price range from query using more robust pattern matching"""
    price_patterns = [
        r'(?:under|below|less than|up to|maximum|max)\s*\$?(\d+,?\d+)',
        r'\$?(\d+,?\d+)\s*(?:and\s*under|or\s*less|or\s*below)',
        r'\$?(\d+,?\d+)\s*-\s*\$?(\d+,?\d+)',
        r'between\s*\$?(\d+,?\d+)\s*and\s*\$?(\d+,?\d+)'
    ]
    
    for pattern in price_patterns:
        matches = re.search(pattern, query.lower())
        if matches:
            if len(matches.groups()) == 1:
                return (0, int(matches.group(1).replace(',', '')))
            elif len(matches.groups()) == 2:
                return (int(matches.group(1).replace(',', '')), int(matches.group(2).replace(',', '')))
    return None

def filter_by_price(listings, price_range):
    """Filter listings based on extracted price range"""
    if not price_range:
        return listings
    
    filtered = []
    price_pattern = r'\$(\d+,?\d+)'
    
    for listing in listings:
        price_match = re.search(price_pattern, listing)
        if price_match:
            price = int(price_match.group(1).replace(',', ''))
            if price_range[0] <= price <= price_range[1]:
                filtered.append(listing)
        elif "cheap" in listing.lower() or "affordable" in listing.lower():
            filtered.append(listing)  # Include cheap/affordable listings even if no price
    
    return filtered if filtered else ["No listings found in that price range"]

def search_web_snippets(query, price_range=None):
    try:
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
        data = json.dumps({"q": query})
        response = requests.post(url, headers=headers, data=data)
        results = response.json().get("organic", [])

        listings = []
        for res in results[:MAX_RESULTS*2]:  # Get more to allow for price filtering
            title = res.get("title", "Listing")
            link = res.get("link", "#")
            snippet = res.get("snippet", "")
            listing_text = f"\U0001F3E0 {title}\n\U0001F4CD [{link}]({link})\n\U0001F4AC {snippet}\n\U0001F3F7 Source: Web"
            listings.append(listing_text)
        
        if price_range:
            listings = filter_by_price(listings, price_range)
        
        return "\n".join(listings[:MAX_RESULTS]) if listings else "No listings found"
    except Exception as e:
        print(f"Search error: {e}")
        return "\U0000274C Web search failed."

# Enhanced city-specific student information
CITY_GUIDES = {
    "miami": {
        "description": "Miami offers diverse student housing options, though prices under $800 are challenging to find near campuses. Consider roommates or areas further from downtown.",
        "universities": [
            "University of Miami (Coral Gables)",
            "Florida International University (Main Campus)",
            "Miami Dade College (Multiple campuses)",
            "Barry University"
        ],
        "transport": [
            "Metrorail to FIU",
            "Metromover (free downtown)",
            "University shuttles",
            "Bike share programs"
        ],
        "leasing_rules": [
            "12-month leases standard",
            "Security deposit (often 1-2 months rent)",
            "Co-signer typically required for students",
            "Limited short-term options"
        ],
        "tips": [
            "Look in Little Havana for more affordable options",
            "Consider becoming an RA for free housing",
            "Check FIU housing boards for roommate shares",
            "Summer sublets can be cheaper"
        ],
        "budget_advice": {
            "under_800": "For under $800, consider: shared rooms, areas like Hialeah or Westchester, or university-affiliated housing subsidies."
        }
    },
    "hartford": {
        "description": "Hartford has relatively affordable student housing options compared to other Northeast cities. The West End and Asylum Hill neighborhoods are popular.",
        "universities": [
            "University of Hartford",
            "Trinity College",
            "Capital Community College",
            "University of Connecticut Hartford Campus"
        ],
        "transport": [
            "CT Transit buses (free with student ID at some schools)",
            "Hartford Line train to nearby cities",
            "Zipcar available on campuses",
            "Walkable downtown area"
        ],
        "leasing_rules": [
            "Security deposit capped at 2 months rent (state law)",
            "9-month leases available near Trinity",
            "Background checks common but flexible for students",
            "July-August best for finding vacancies"
        ],
        "tips": [
            "Parkville neighborhood popular with art students",
            "West End has historic homes converted to student housing",
            "Look for 'student-friendly' labeled apartments",
            "Winter months may have better pricing"
        ],
        "budget_advice": {
            "under_700": "At under $700, look for: shared apartments in Frog Hollow, efficiency units in downtown, or university housing waitlists."
        }
    }
}

def get_city_guide(city, price_range=None):
    """Get city-specific student information with budget context"""
    city = city.lower()
    guide = CITY_GUIDES.get(city, {})
    
    if not guide:
        return """
\U0001F393 **General Student Housing Advice**

\U0001F4CC **When searching for affordable housing:**
• Consider shared rooms or apartments
• Look further from campus (check transit routes)
• Ask about university housing subsidies
• Check for income-restricted housing programs

\U0001F3DB **Common Strategies:**
• Become an RA for free/discounted housing
• Look for live-in caregiver positions
• Consider sublets or short-term rentals
• Check religious organizations for housing help
"""
    
    # Build the guide response
    response = f"""
\U0001F393 **Student Housing Guide for {city.capitalize()}**

\U0001F4CC **Key Information:**
{guide.get('description', 'General student housing information.')}
"""
    
    # Add budget-specific advice if available
    if price_range:
        budget_key = f"under_{price_range[1]}" if price_range[1] < 1000 else "affordable"
        budget_advice = guide.get("budget_advice", {}).get(budget_key)
        if budget_advice:
            response += f"\n\U0001F4B5 **Budget Tips ({price_range[1]}):**\n{budget_advice}\n"

    response += f"""
\U0001F3DB **Nearby Universities:"""
    for uni in guide.get("universities", ["Check local colleges and universities"]):
        response += f"\n• {uni}"

    response += f"""

\U0001F68C **Transportation Options:"""
    for transport in guide.get("transport", ["Public transport available"]):
        response += f"\n• {transport}"

    response += f"""

\U0001F4DD **Leasing Details:"""
    for lease in guide.get("leasing_rules", ["Typical lease terms apply"]):
        response += f"\n• {lease}"

    response += f"""

\U0001F4A1 **Pro Tips:"""
    for tip in guide.get("tips", ["Start your search early"]):
        response += f"\n• {tip}"

    response += """

\U0001F50D **Additional Advice:**
• Always verify listings in person
• Check for student housing fraud warnings
• Ask about utility costs upfront
• Consider renters insurance

\U0001F4DE **Next Steps:"""
    steps = [
        f"1. Contact {guide.get('universities', ['university'])[0]} housing office",
        "2. Search for student housing groups on Facebook",
        "3. Check university off-campus housing portals"
    ]
    response += "\n" + "\n".join(steps)
    
    return response

def handle_user_query(message, chat_history=None):
    # Extract price range
    price_range = extract_price_range(message)
    
    # Extract city
    city = None
    city_patterns = [
        r'in\s([a-zA-Z\s]+?)(?:\sunder|\sbelow|\sfor|\snear|$)',
        r'near\s([a-zA-Z\s]+)',
        r'at\s([a-zA-Z\s]+)',
        r'around\s([a-zA-Z\s]+)'
    ]
    
    for pattern in city_patterns:
        match = re.search(pattern, message.lower())
        if match:
            city = match.group(1).strip()
            break
    
    # Create search query
    base_query = f"site:rent.com {message}"
    listings = search_web_snippets(base_query, price_range)
    
    # Get city guide if city found
    city_guide = get_city_guide(city, price_range) if city else ""
    
    # Compose response
    response = f"\U0001F50D Housing Listings for: {message}\n\n"
    response += listings if listings else "No current listings found"
    
    if city_guide:
        response += "\n\n" + city_guide
    
    return response

with gr.Blocks() as app:
    gr.Markdown("""
    # \U0001F3E0 Roamio - Student Housing Finder
    *Now with accurate price filtering and city-specific advice*
    """)
    gr.ChatInterface(
        fn=handle_user_query,
        examples=[
            "Find apartments in Miami under $800",
            "Show houses in Hartford below $700",
            "Student housing near University of Miami",
            "Affordable housing in Boston for students"
        ],
        title="Student Housing Search"
    )

app.launch()

/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7876
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://8e6cd73a038af5e38e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
